In [ ]:
'''       Pandas.DataFRame       '''

import pandas as pd


expenses = [156, 130, 270]
income = [478, 512, 196]
years = [2018, 2019, 2020]

def create_companyDF(income, expenses, years):
    """
    Создайте функцию create_companyDF(income, expenses, years), которая  возвращает DataFrame, 
    составленный из входных данных со столбцами “Income” и “Expenses” и индексами, соответствующим годам рассматриваемого периода.
    """
    df = pd.DataFrame({
        'Income':income,
        'Expenses':expenses,
    })
    df.index=years
    return df

def get_profit(df, year):
    
    """
    А также напишите функцию get_profit(df, year), которая возвращает разницу между доходом и расходом, записанных в таблице df, за год year.
    Учтите, что если информация за запрашиваемый год не указана в вашей таблице вам необходимо вернуть None. 
    """
    try:
       profit=df.loc[year, 'Income'] - df.loc[year, 'Expenses']
    except KeyError:
        return None
    return profit

if __name__ == '__main__':
    expenses = [156, 130, 270]
    income = [478, 512, 196]
    years = [2018, 2019, 2020]
    
    scienceyou = create_companyDF(income, expenses, years)
    print(get_profit(scienceyou, 2020)) #-74


-74


In [56]:
import pandas as pd
import numpy as np

melb_data = pd.read_csv('data/melb_data_ps.csv', sep=',')
#melb_data.head()

melb_df = melb_data.copy()   #  делаем копию, чтобы не факапить изначальный ДатаФрейм. Далее в коде стараемся создавать новую копию для новых значимых операций, чтобы было легко вернуться в случае чего
#melb_df.head()
melb_df = melb_df.drop(['index', 'Coordinates'], axis=1)  # дропаем соответсвующие столбцы
#melb_df.head()
total_rooms = melb_df['Rooms'] + melb_df['Bedroom'] + melb_df['Bathroom'] # над столбцами можно производить мат.операции, они поэлементны. В данном случае суммируем кол-во вообще всех комнат в объекте недвиги
#display(total_rooms)
melb_df['MeanRoomsSquare'] = melb_df['BuildingArea'] / total_rooms # Вводим новый признак "средняя площадь комнаты"
#display(melb_df['MeanRoomsSquare'])
diff_area = melb_df['BuildingArea'] - melb_df['Landsize']
sum_area = melb_df['BuildingArea'] + melb_df['Landsize']
melb_df['AreaRatio'] = diff_area/sum_area               # Вводим новый признак, показывающий соотношение площади недвиги и земли вокруг. если  -1 то дома нет(только земля), если 1 то нет земли вокруг.
#display(melb_df['AreaRatio'])
melb_df['Date'] = pd.to_datetime(melb_df['Date'], dayfirst=True)  # преобразование даты в datetime
#display(melb_df['Date'])

years_sold = melb_df['Date'].dt.year        # обратившись по атрибуту dt.year в столбце Date, мы можем «достать» год продажи
#print(years_sold)
#print('Min year sold:', years_sold.min())
#print('Max year sold:', years_sold.max())
#print('Mode year sold:', years_sold.mode()[0])

melb_df['MonthSale'] = melb_df['Date'].dt.month  # Определяем месяц с наибольшим процентом продаж относительно общего числа
melb_df['MonthSale'].value_counts(normalize=True)

delta_days = melb_df['Date'] - pd.to_datetime('2016-01-01') # вычисляем, сколько дней прошло с 1 января 2016 года до момента продажи объекта
#display(delta_days)




melb_df['WeekdaySale'] = melb_df['Date'].dt.dayofweek
weekend=melb_df['WeekdaySale'].value_counts()
#weekend_count = weekend[5] + weekend[6]
#print(weekend)

#print(melb_df['WeekdaySale'].loc[177])


def get_weekend(weekday):

    if weekday==5 or weekday==6:
        return 1
    else:
        return 0

melb_df['Weekend']=melb_df['WeekdaySale'].apply(get_weekend)   # создали новый признак Weekend(true or false)
melb_df[melb_df['Weekend']==1]['Price'].mean()                 # и посчитали среднюю стоимость



popular_sellers=melb_df['SellerG'].value_counts().nlargest(49).index     # 49 наиболее поплулярных продавцов

melb_df['SellerG']=melb_df['SellerG'].apply(lambda x: x if x in popular_sellers else 'other')

#print(melb_df[melb_df['SellerG']=='Nelson']['Price'].min()/melb_df[melb_df['SellerG']=='other']['Price'].min())


'''         Определяем уникальные категории         '''
# создаём пустой список
unique_list = []
# пробегаемся по именам столбцов в таблице
for col in melb_df.columns:
    # создаём кортеж (имя столбца, число уникальных значений)
    item = (col, melb_df[col].nunique(),melb_df[col].dtype) 
    # добавляем кортеж в список
    unique_list.append(item) 
# создаём вспомогательную таблицу и сортируем её
unique_counts = pd.DataFrame(
    unique_list,
    columns=['Column_Name', 'Num_Unique', 'Type']
).sort_values(by='Num_Unique',  ignore_index=True)
# выводим её на экран
#display(unique_counts)


'''         Category         '''

cols_to_exclude = ['Date', 'Rooms', 'Bedroom', 'Bathroom', 'Car'] # список столбцов, которые мы не берём во внимание
max_unique_count = 150 # задаём максимальное число уникальных категорий
for col in melb_df.columns: # цикл по именам столбцов
    if melb_df[col].nunique() < max_unique_count and col not in cols_to_exclude: # проверяем условие
        melb_df[col] = melb_df[col].astype('category') # преобразуем тип столбца
#display(melb_df.info())

popular_urb=melb_df['Suburb'].value_counts().nlargest(119).index
melb_df['Suburb']=melb_df['Suburb'].apply(lambda x: x if x in popular_sellers else 'other')
melb_df['Suburb']=melb_df['Suburb'].astype('category')
melb_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 26 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Suburb           13580 non-null  category      
 1   Address          13580 non-null  object        
 2   Rooms            13580 non-null  int64         
 3   Type             13580 non-null  category      
 4   Price            13580 non-null  float64       
 5   Method           13580 non-null  category      
 6   SellerG          13580 non-null  category      
 7   Date             13580 non-null  datetime64[ns]
 8   Distance         13580 non-null  float64       
 9   Postcode         13580 non-null  int64         
 10  Bedroom          13580 non-null  int64         
 11  Bathroom         13580 non-null  int64         
 12  Car              13580 non-null  int64         
 13  Landsize         13580 non-null  float64       
 14  BuildingArea     13580 non-null  float

In [37]:
import pandas as pd

usa_ufo = pd.read_csv('https://raw.githubusercontent.com/justmarkham/pandas-videos/master/data/ufo.csv')
usa_ufo['Time'] = pd.to_datetime(usa_ufo['Time'])
#usa_ufo['Time'].dt.year.value_counts()
delta_days=usa_ufo[usa_ufo['State']=='NV']['Time'].dt.date.diff()       #  интервал между случаями наблюдения НЛО в Неваде
print(delta_days.dt.days.mean())                                        #  приводим к дням и вычисляем среднее значение


68.92932862190813


In [1]:
import pandas as pd

experience_col = pd.Series([
        'Опыт работы 8 лет 3 месяца',
        'Опыт работы 3 года 5 месяцев',
        'Опыт работы 1 год 9 месяцев',
        'Опыт работы 3 месяца',
        'Опыт работы 6 лет'
        ])

def get_experience(arg):                # перерабатываем годы и месяцы опыта в месяцы
    ym=arg.split(' ')
    y=0
    m=0
    if len(ym)>4:
        y= ym[2]
        m= ym[4]
    elif ym[3]=='месяца' or ym[3]=='месяцев' or ym[3]=='месяц':
        m =ym[2]
    else:
        y= ym[2]
    
    y=int(y)
    m=int(m)    
    res=y*12+m
    return res

experience_month = experience_col.apply(get_experience)
print(experience_month)

0    99
1    41
2    21
3     3
4    72
dtype: int64


In [1]:
import pandas as pd

rides=pd.read_csv('./data/citibike-tripdata.csv')

#rides['start station id'].nunique()
#rides['end station id'].nunique()
#rides['end station name'].value_counts()

rides=rides.drop(['start station id', 'end station id'], axis=1)

def get_age(year):
    age=2018-year
    return age
rides['age']= rides['birth year'].apply(get_age)
rides=rides.drop(['birth year'], axis=1)

rides['old folks']=rides['age'].apply(lambda x: x if x>60 else None)

#rides['starttime']=pd.to_datetime(rides['starttime'])

#rides.info()


start_trip=pd.to_datetime(rides['starttime'])
end_trip=pd.to_datetime(rides['stoptime'])

rides['trip duraton']= end_trip - start_trip

rides['trip duraton'].mean().seconds

rides['starttime']=pd.to_datetime(rides['starttime'])
rides['days of week'] = rides['starttime'].dt.dayofweek

def get_weekend(weekday):

    if weekday==5 or weekday==6:
        return 1
    else:
        return 0

rides['weekend'] = rides['days of week'].apply(get_weekend)
rides['weekend'].value_counts()



def get_daytime(time):
    if 0<=time<=6:
        return 'night'
    elif 6<time<=12:
        return 'morning'
    elif 12<time<=18:
        return 'day'
    elif 18<time<=23:
        return 'evening'
rides['time_of_day'] = rides['starttime'].dt.hour.apply(get_daytime)

len(rides[rides['time_of_day']=='day'])/len(rides[rides['time_of_day']=='night'])


9.480411004308916